# I. Monte-Carlo for some multiplicative free convolutions.

In [ ]:
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt

N=1000
num_bins=20

Scenario = "Case2"

# Initiate scenario
import freeDeconvolution

c, p, population_spectrum, interval_cdf, population_cdf = freeDeconvolution.scenarios.initiate_scenario( Scenario, N)
r = (1+np.sqrt(c))**2 #Right end of MP
l = (1-np.sqrt(c))**2 #Left end of MP
print("c :", c)

freeDeconvolution.plots.plot_population( population_spectrum, num_bins, interval_cdf, population_cdf, plt )

# Sample
print( "Sampling... ")
diag = freeDeconvolution.sampling.sample_wishart( p, N, population_spectrum )

freeDeconvolution.plots.plot_observed_spectrum( Scenario, c, diag, num_bins, plt)

# II. Cauchy-Stieljes transform: Empirical vs Theoretical

For convenience, let us compute:
$$ G(z) = \int_\mathbb{R} \frac{\mu_{MP}(dt)}{z-t}$$

The formula is
$$ G(z) = \frac{z-1+c - \sqrt{ (z-1-c)^2 - 4c} }{2z \ c}$$
In particular
$$ M(z) = z G(z) - 1 = \frac{1}{2c} ( z-1-c - \sqrt{ (z-1-c)^2 - 4c} )$$
which solves the quadratic equation:
$$ c*m^2 - (z-1-c)*m + 1 = 0$$
or equivalently
$$ m z = m(1 + c) + c*m^2 + 1
       = c(m+1)(m+1/c)
$$
or equivalently
$$
   s = (1+m)/(m*z)
     = 1/(cm+1)
$$


In [ ]:
# Warning: This has changed to fit El Karoui's Case 1
def G_theoretical(z):
    sqrt = np.sqrt( (z-c-1)*(z-c-1)-4*c )
    positive_imag = (np.imag(sqrt)>0)
    sqrt = positive_imag*sqrt - (1-positive_imag)*sqrt
    return (z+c-1-sqrt)/(2*z*c)

mu_observed = freeDeconvolution.core.DiscreteMeasure( diag, None)
mu_signal   = freeDeconvolution.core.DiscreteMeasure( population_spectrum, None)

if Scenario=='Case1':
    interval = np.linspace(-1, 3, 100)
    density1 = -np.imag( G_theoretical(interval+0.05j) )/np.pi
    density2 = -np.imag( mu_observed.G_empirical(interval+0.05j) )/np.pi
    plt.plot( interval, density1, label='Theoretical')
    plt.plot( interval, density2, label='Empirical')
    plt.legend()
    plt.show()

mu_observed.compute_second_kind()

zeroes_first_kind   = mu_observed.zeroes_first_kind
zeroes_second_kind  = mu_observed.zeroes_second_kind

## III. Find bounding box

In [ ]:
degree = len(diag)

print("Eigenvalues")
print("min: ", np.min(diag))
print("max: ", np.max(diag))

mesh_size = 10000
radius = np.max(diag)/2 + 1
center = np.max(diag)/2
interval = np.linspace(0, 2*np.pi, mesh_size)
contour = center + radius*( np.cos(interval) + np.sin(interval)*1.0j)
plt.scatter( np.real(diag), np.imag(diag), c='r')
plt.plot( np.real(contour), np.imag(contour) )
plt.show()

def index_integrand(z):
    values = mu_observed.Markov_Krein_prime(z)/mu_observed.Markov_Krein(z)
    return values

values = index_integrand(contour)
dz = 1.0j*(contour-center)*2*np.pi/(mesh_size) 
index  = np.sum(dz*values)/(2*np.pi*1.0j)
print( "Index: ", index)
print( "Root count: ", index+2*degree)

In [ ]:
from freeDeconvolution import boxes

print( "Box segments enumeration: ")
print( boxes.box_segments_enum )

def compute_index( box, mesh_size, plot=True, color='b'):
       interval =  np.linspace( 0,1, mesh_size)
       integral = 0
       for segment in boxes.box_segments_enum:
              vector = box[ segment[1] ] - box[ segment[0] ]
              origin = box[ segment[0] ]
              s = origin + interval*vector
              #
              values = index_integrand(s)
              dz = ( s[-1]-s[0] )/mesh_size
              integral = integral + np.sum( values*dz )
       return integral/(2*np.pi*1.0j)

In [ ]:
# TODO: Make it more versatile. Here tuning by hand of radius.
print("Finding bounding box...")

radius = 1
mesh_size = int(1e4)
bounding_box = {
       'top_left'    : np.min(diag) - 0.5 + radius*1.0j,
       'bottom_right': np.max(diag) + 0.5 - radius*1.0j,
}
bounding_box   = boxes.extend_box(bounding_box)
index = compute_index( bounding_box, mesh_size)
index = np.real(index+2*degree)
root_count = int( np.round( index ) )
error = index-root_count
print( "Index     : ", index)
print( "Root count: ", "2x", 0.5*root_count)
print( "p         : ", p)
print( "")

# If false, the bounding box missed roots
assert( p-1 == int(0.5*root_count))

In [ ]:
box = bounding_box.copy()
box['bottom_left'] = box['top_left'] # For initialization, bottom_left needs to be the previous top_left
radius = box['height']/2
stop_at_first_nonempty = True        # Stop at first found box with roots

# Loop for multiple passes and more
boxes_with_roots = []
root_counter = 0
total_roots  = p-1 # Total number of roots in upper half plane
i = 0
while( root_counter < total_roots ):
       i = i+1
       print(f"Pass {i}:")
       radius = radius/2
       box = {
              'top_left'    : box['bottom_left'],
              'bottom_right': np.real( box['bottom_right'] ) + radius*1.0j,
       }
       box   = boxes.extend_box(box)
       index = compute_index( box, mesh_size, plot=False)
       index = np.real(index)
       root_count = int( np.round( index ) )
       root_counter = root_counter + root_count
       error = index-root_count
       print( "Index: ", index)
       print( "Root count / Total: ", root_count, '/', root_counter)
       print( "Found:", root_counter, " / ", total_roots )
       print( "")
       #
       if root_count>0:
              box['root_count'] = root_count
              boxes_with_roots.append( box )
              if stop_at_first_nonempty:
                     break
#

In [ ]:
print("There are ", len(boxes_with_roots), "large boxes containing zeroes.")

# Plotting
for box in boxes_with_roots:
       color = np.random.rand(3)
       boxes.plot_box( box, mesh_size, color, plt)
plt.scatter( np.real(diag), np.imag(diag), c='r')
plt.show()

In [ ]:
box = {
        'top_left'    : boxes_with_roots[0]['top_left'],
        'bottom_right': np.real( boxes_with_roots[0]['bottom_right'] ),
}
box['bottom_right'] = box['bottom_right']-1.0j*np.imag(box['top_left'])
box = boxes.extend_box(box)

# Plotting
color = np.random.rand(3)
boxes.plot_box( box, mesh_size, color, plt)
plt.scatter( np.real(diag), np.imag(diag), c='r')
plt.show()

bounding_box = box

## IV. Contour integrals

In [ ]:
contour_type = "circle"
mesh_size    = int(2e5)

if contour_type=="rectangle":
    path = boxes.box_to_path( bounding_box, mesh_size )
    z_array = np.array( path )
elif contour_type=="circle":
    a = np.min(zeroes_first_kind)
    b = np.max(zeroes_first_kind)
    center = (a+b)/2
    radius = np.maximum( 1.1*(b-a)/2 , bounding_box["height"] )
    #
    interval = np.linspace(0, 2*np.pi, mesh_size)
    z_array = center + radius*( np.cos(interval) + np.sin(interval)*1.0j)
else:
    raise ValueError()

In [ ]:
m_array = mu_observed.M_empirical( z_array )

# Plots in z and m space
fig, axs = plt.subplots(1, 2, figsize=(20, 10), sharey=False)

axs[0].plot( np.real(z_array), np.imag(z_array), label=f'Lift in z space)')
axs[0].scatter( np.real(zeroes_first_kind), np.imag(zeroes_first_kind), marker='*', c='r', label=f'Eigenvalues (p={p})')
axs[0].scatter( np.real(zeroes_second_kind), np.imag(zeroes_second_kind), marker='*', c='b', label=f'Zeroes of the second kind (p-1={p-1})')
axs[0].set_title( 'z space')

axs[1].plot( np.real(m_array), np.imag(m_array), label=f'Path in m space)')
axs[1].set_title( 'm space')
axs[1].set_ylim( -5, 5)

plt.legend()
plt.show()

In [ ]:
s_array = (1+m_array)/(m_array*z_array)

if Scenario=='Case1':
    c_interval = np.linspace(0.1, 0.5, 100)

    error_array = []
    for c_ in c_interval:
        error = np.abs( s_array*(c_*m_array+1)-1).max()
        error_array.append( error )
    error_array = np.array( error_array )
    plt.plot( c_interval, error_array )
    plt.show()

### Formulas

$$ m = zg - 1$$
$$ s = \frac{1+m}{m z}
   \Leftrightarrow
   sz - 1 = \frac{1}{m}
$$

In [ ]:
print( "c: ", c)

s_signal_array = s_array
s_noise_array  = 1/(c*m_array + 1)
s_deconv_array = s_signal_array/s_noise_array
m_deconv_array = 1/( s_deconv_array*z_array - 1)
m_deconv_theoretical_array = mu_signal.M_empirical( z_array )

# Plots in z and m space
fig, axs = plt.subplots(1, 2, figsize=(20, 10), sharey=False)

axs[0].plot( np.real(z_array), np.imag(z_array), label=f'Lift in z space)')
axs[0].scatter( np.real(zeroes_first_kind), np.imag(zeroes_first_kind), marker='*', c='r', label=f'Eigenvalues (p={p})')
axs[0].scatter( np.real(zeroes_second_kind), np.imag(zeroes_second_kind), marker='*', c='b', label=f'Zeroes of the second kind (p-1={p-1})')
axs[0].set_title( 'z space')

axs[1].plot( np.real(m_deconv_array), np.imag(m_deconv_array), label=f'Empirical path in m space')
axs[1].plot( np.real(m_deconv_theoretical_array), np.imag(m_deconv_theoretical_array), label=f'Ground truth in m space')
axs[1].set_title( 'm space')
axs[1].set_ylim( -7, 7)

plt.legend()
plt.show()

# IV. Deconvolution 

In [ ]:
g_deconv_array = (m_deconv_array+1)/z_array
g_deconv_theoretical = (m_deconv_theoretical_array+1)/z_array

if contour_type=='rectangle':
    resolution = 2/bounding_box['height']
elif contour_type=='circle':
    resolution = 1.0/4
dz_array = z_array-np.roll(z_array, shift=1)
empirical_cdf = []
for x in interval_cdf:
    sigmoid = 1/(1+np.exp(resolution*(z_array-x)))
    value = g_deconv_array*sigmoid*dz_array
    value = value.sum()/(2*np.pi*1.0j)
    empirical_cdf.append( value )
# end for
empirical_cdf = np.real( np.array( empirical_cdf ) )

smoothed_population_cdf = []
for x in interval_cdf:
    sigmoid = 1/( 1 + np.exp(resolution*(population_spectrum-x)) )
    value = sigmoid.mean()
    smoothed_population_cdf.append( value )
# end for
smoothed_population_cdf = np.array( smoothed_population_cdf )


plt.figure( figsize=(10,5) )
plt.plot( interval_cdf, population_cdf, label='Asymptotic ground truth = Theoretical cdf of deconvolved signal' )
plt.plot( interval_cdf, empirical_cdf, label=f'''Empirical cdf of deconvolved signal and blur at resolution {1/resolution}''' )
plt.plot( interval_cdf, smoothed_population_cdf, label='Theoretical cdf of deconvolved signal at same resolution' )
plt.title( "CDF")
plt.legend()
plt.show()

def array_increment(array):
    return array[1:]-array[:-1]

plt.figure( figsize=(10,5) )
plt.plot( interval_cdf[:-1], array_increment(population_cdf), label='Asymptotic ground truth = Theoretical density of deconvolved signal' )
plt.plot( interval_cdf[:-1], array_increment(empirical_cdf), label=f'''Empirical density of deconvolved signal and blur at resolution {1/resolution}''' )
plt.title( "Densities")
plt.legend()
plt.show()

# V. OPRL reconstruction

In [ ]:
dz_array  = z_array-np.roll(z_array, shift=1)
def cauchy_integral_g_deconv( f ):
    value = g_deconv_array*f*dz_array
    #value = g_deconv_theoretical*f*dz_array
    return value.sum()/(2*np.pi*1.0j)

## V. 1. Direct approach: Cholesky

In [ ]:
# Compute moments
# Input: z_array, g_deconv_array

moments_count = 10
mom_array = np.zeros( 2*moments_count )
for mom_index in range(2*moments_count):
    value = cauchy_integral_g_deconv( z_array**mom_index )
    mom_array[ mom_index ] = np.real(value)
# end for
print( mom_array )

In [ ]:
# Form moment matrix
from scipy.linalg import cholesky as scipy_cholesky

mom_matrix = np.zeros( shape=(moments_count, moments_count) )
for index in range( moments_count ):
    mom_matrix[index,:] = mom_array[index:(index+moments_count)]

# Cholesky
try:
    cholesky = scipy_cholesky( mom_matrix )
    print("Passed!")
    print("")
except np.linalg.LinAlgError as err:
    print( err  )
    print("")

# Retry
mom_count = 5
print(f'''Retrying with {mom_count}...''')
try:
    cholesky = scipy_cholesky( mom_matrix[0:mom_count, 0:mom_count], lower=False )
    print("Passed")
except np.linalg.LinAlgError as err:
    print( err  )

print( cholesky )


In [ ]:
def jacobi_indices( dim ):
    diagonal_indices   = np.array( np.diag_indices( dim ) )
    extra_diag_indices = diagonal_indices.copy()[:, :-1]
    extra_diag_indices[1,:] += 1
    return diagonal_indices, extra_diag_indices


mom_count = cholesky.shape[0]
diag_indices, extra_indices = jacobi_indices( mom_count )
print( diag_indices )
print( extra_indices )
diag_cholesky = cholesky[diag_indices[0,:] , diag_indices[1,:]]
extra_diag    = cholesky[extra_indices[0,:], extra_indices[1,:]]
print( diag_cholesky )
print( extra_diag )
print("")

# Compute Jacobi
jacobi_b = diag_cholesky[1:]/diag_cholesky[:-1]
jacobi_a = np.zeros_like( diag_cholesky )
jacobi_a[0] = diag_cholesky[0]
x_over_y = extra_diag/diag_cholesky[1:]
jacobi_a[1:-1] = x_over_y[1:]-x_over_y[:-1]
# Print Jacobi coefficients
print( "Jacobi coefficients:")
print( "b: ", jacobi_b )
print( "a: ", jacobi_a )
print( "")

# Form Jacobi matrix
jacobi = np.zeros( shape=(mom_count, mom_count) )
jacobi[extra_indices[0,:], extra_indices[1,:]] = jacobi_b
jacobi = jacobi + jacobi.T + np.diag( jacobi_a )
print( "Jacobi matrix ")
print( jacobi )
print( "" )

# Eigenvalues
print("Eigenvalues")
eigen, vectors = np.linalg.eig(jacobi)
eigen = np.sort(eigen)
print(eigen)
# Compute weights
vandermonde = np.vander( eigen ).T
weights = np.linalg.solve( vandermonde, mom_array[0:mom_count] )
print( "Raw Weights: ")
print( weights)

In [ ]:
# Discard negative weights and eigenvalues outside of support
min_zero = np.min( zeroes_first_kind )
max_zero = np.max( zeroes_first_kind )
admissible_eigen   = np.where( (eigen >= min_zero) *  (eigen <= max_zero) * (weights > 0) )[0]
admissible_weights = weights[ admissible_eigen ]
admissible_weights = admissible_weights/np.sum( admissible_weights )
admissible_eigen   = eigen[ admissible_eigen ]
print("Admissible weights: ", admissible_weights)
print("Admissible eigen  : ", admissible_eigen)

# Plot
space_array   = np.linspace(-1, 7, 100)
empirical_cdf = np.zeros_like( space_array )
for index in range(len(admissible_eigen)):
    eigenvalue = admissible_eigen[ index ]
    empirical_cdf += admissible_weights[index] * ( interval_cdf >= eigenvalue )
# end for
empirical_cdf = np.array( empirical_cdf )

plt.figure( figsize=(10,5) )
plt.plot( interval_cdf, population_cdf, label='Asymptotic ground truth = Theoretical cdf of deconvolved signal' )
plt.plot( interval_cdf, np.real(empirical_cdf), label=f'''Empirical cdf of deconvolved signal with OPRL reconstruction''' )
plt.legend()
plt.show()


## V. 2. Iterative approach: Three term recurrence

In this approach, we avoid computing moments at all cost!

For the monic polynomials $P_n$:
$$ X P_n = P_{n+1} + a_n P_n + b_{n-1}^2 P_{n-1} $$

From that one deduces (after some work) that:
$$ \| P_n \| = b_1 \dots b_{n-1}, $$
which yields the recurrence for orthonormal polynomials $p_n$:
$$ X p_n = b_n p_{n+1} + a_n p_n + b_{n-1} p_{n-1} $$

In [ ]:
def polynomial_eval( coeffs, points):
    m = len(coeffs)
    powers = np.arange( 0, m, 1)
    vander = points[..., None]**powers[None, ...]
    return np.dot(vander, coeffs)

# Test
points = np.arange( 0, 5)
coefficients = [1, 0]
print( polynomial_eval(coefficients, points) )
coefficients = [1, 1]
print( polynomial_eval(coefficients, points) )


In [ ]:
# Compute OPRL
polynomials = []
norms = []

# First in the iteration: P_0 = 1
polynomials.append( [1] )
norms.append( 1 )

# Second in the iteration: P_1 = X-m_1
m1 = np.real( cauchy_integral_g_deconv( z_array ) )
polynomials.append( [-m1, 1] )
P1 = np.array( polynomials[-1] )
values_P1 = polynomial_eval(P1, z_array)
norm2_P1 = np.real( cauchy_integral_g_deconv( values_P1*values_P1 ) )
norm_P1  = np.sqrt(norm2_P1)
assert( norm2_P1 > 0)
norms.append( norm_P1 )

# Apply three term recurrence
OPRL_order = 10
jacobi_a = [ m1 ]
jacobi_b = [ ]
for n in np.arange(1, OPRL_order, 1):
    Pn  = polynomials[-1] + [0.0]
    XPn = [0.0] + polynomials[-1]
    Qn  = polynomials[-2] + [0.0, 0.0] # Short for P_{n-1}
    Pn, XPn, Qn = [ np.array(x) for x in [Pn, XPn, Qn] ]
    # Values along path
    values_Pn  = polynomial_eval(  Pn, z_array)
    values_XPn = z_array*values_Pn
    values_Qn  = polynomial_eval(  Qn, z_array)
    # Norms
    norm_Pn = norms[-1]
    norm_Qn = norms[-2]
    # Compte a_n and b_{n-1}
    a  = np.real( cauchy_integral_g_deconv( values_XPn*values_Pn ) )/( norm_Pn)
    b2 = np.real( cauchy_integral_g_deconv( values_XPn*values_Qn ) )/( norm_Qn)
    print( "n: ", n+1)
    print( "a: ", a)
    print( "b2: ", b2)
    print( "")
    if b2 < 0:
        print( "Defaulting at n =", n+1)
        print( "")
        OPRL_order = n
        #jacobi_a.append( a )
        break
    #
    b = np.sqrt(b2)
    jacobi_a.append( a )
    jacobi_b.append( b )
    norms.append( norm_Pn*b )
    # Three term recurrence
    new_P = XPn - a*Pn - b*b*Qn
    polynomials.append( list(new_P) )
# end for
print( "Jacobi coefficients:")
print( jacobi_a )
print( jacobi_b )
print( "")
print( "Norms:")
print( norms )

In [ ]:
# Form Jacobi matrix
jacobi = np.zeros( shape=(OPRL_order, OPRL_order) )
diag_indices, extra_indices = jacobi_indices( OPRL_order )
jacobi[extra_indices[0,:], extra_indices[1,:]] = jacobi_b
jacobi = jacobi + jacobi.T + np.diag( jacobi_a )
print( "Jacobi matrix ")
print( jacobi )
print( "" )

# Eigenvalues
print("Eigenvalues")
eigen, vectors = np.linalg.eig(jacobi)
eigen = np.sort(eigen)
print(eigen)

# Check: Zeros of last OPRL?
print("Evaluation on last OPRL")
last_P = polynomials[-1]
values = polynomial_eval( last_P, eigen)
print( values )


In [ ]:
Q_polynomials = []

Q0 = [0.0]
Q_polynomials.append( Q0) 

Q1 = [1.0, 0.0]
Q_polynomials.append( Q1) 

for n in np.arange(1, OPRL_order, 1):
    Qn  = Q_polynomials[-1] + [0.0]
    XQn = [0.0] + Q_polynomials[-1]
    Rn  = Q_polynomials[-2] + [0.0, 0.0]
    Qn, XQn, Rn = [ np.array(x) for x in [Qn, XQn, Rn] ]
    #
    a = jacobi_a[n]
    if n == len(jacobi_b):
        b = 0
    else:
        b = jacobi_b[n]
    # Three term recurrence
    new_Q = XQn - a*Qn - b*b*Rn
    # Append
    new_Q = list( new_Q )
    Q_polynomials.append( new_Q)

#
print( len(polynomials) )
print( len(Q_polynomials) )

In [ ]:
# Compute weights
last_Q = Q_polynomials[-1]
last_P = polynomials[-1]
print( last_P )
last_P_prime = np.arange( 0, len(last_P) )*last_P
last_P_prime = last_P_prime[1:]
print( last_P_prime )
weights = polynomial_eval(last_Q, eigen)/polynomial_eval(last_P_prime, eigen)
weights = weights/weights.sum()
print( "Support: ")
print( eigen)
print( "Raw Weights: ")
print( weights)

In [ ]:
# Plot
empirical_cdf = np.zeros_like( space_array )
for index in range(len(eigen)):
    eigenvalue = eigen[ index ]
    empirical_cdf += weights[index] * ( interval_cdf >= eigenvalue )
# end for
empirical_cdf = np.array( empirical_cdf )

plt.figure( figsize=(10,5) )
plt.plot( interval_cdf, population_cdf, label='Asymptotic ground truth = Theoretical cdf of deconvolved signal' )
plt.plot( interval_cdf, np.real(empirical_cdf), label=f'''Empirical cdf of deconvolved signal with OPRL reconstruction''' )
plt.legend()
plt.show()

In [ ]:
# Discard negative weights and eigenvalues outside of support
min_zero = np.min( zeroes_first_kind )
max_zero = np.max( zeroes_first_kind )
admissible_eigen   = np.where( (eigen >= min_zero) *  (eigen <= max_zero) * (weights > 0) )[0]
admissible_weights = weights[ admissible_eigen ]
admissible_weights = admissible_weights/np.sum( admissible_weights )
admissible_eigen   = eigen[ admissible_eigen ]
print("Admissible weights: ", admissible_weights)
print("Admissible eigen  : ", admissible_eigen)

# Plot
empirical_cdf = np.zeros_like( space_array )
for index in range(len(admissible_eigen)):
    eigenvalue = admissible_eigen[ index ]
    empirical_cdf += admissible_weights[index] * ( interval_cdf >= eigenvalue )
# end for
empirical_cdf = np.array( empirical_cdf )

plt.figure( figsize=(10,5) )
plt.plot( interval_cdf, population_cdf, label='Asymptotic ground truth = Theoretical cdf of deconvolved signal' )
plt.plot( interval_cdf, np.real(empirical_cdf), label=f'''Empirical cdf of deconvolved signal with OPRL reconstruction''' )
plt.legend()
plt.show()